In [ ]:
import os
import random
from PIL import Image, ImageOps, ImageDraw
import time
import math
import shutil

def intersect(box1, box2):
    return not (box1[2] <= box2[0] or box1[0] >= box2[2] or box1[3] <= box2[1] or box1[1] >= box2[3])

def transform(image):
    
    angle = random.uniform(-30, 30)
    # Randomly rotate the image
    image = image.rotate(angle, resample=Image.BICUBIC)

     # Randomly flip the image
    if random.random() > 0.5:
        image = ImageOps.mirror(image)
        
    return image

def lblformat(box, background_size):
    labelimg_box = [
    (box[0]/ float(background_size)) + ((box[2]-box[0])/ float(background_size))*0.5,#(box1[0]/ float(background_size))*0.5,  # xmin
    (box[1]/ float(background_size))+ ((box[3]-box[1])/ float(background_size))*0.5,#(box1[1]/ float(background_size))*0.5,  # ymin
    (box[2]-box[0])/ float(background_size),  # width
    (box[3]-box[1])/ float(background_size),  # height
    ]
    return labelimg_box

def detect_class(name):
    if 'cardboard' in name:
        return 0
    elif 'glass' in name:
        return 1
    elif 'metal' in name:
        return 2
    elif 'paper' in name:
        return 3
    elif 'plastic' in name:
        return 4
    else:
        return 5

In [ ]:
background_size = 800

directory = 'png_train'
output_dir = "new_images_train"

folder_count = len([item for item in os.listdir(output_dir) if os.path.isdir(os.path.join(output_dir , item))])

folder_name = f'exp{folder_count+1}'

new_folder = os.path.join(output_dir, f'exp{folder_count+1}')
os.mkdir(new_folder)
output_dir = new_folder

#copy the classes.txt file to the new folder
source_file = '/Users/casper/Desktop/yolov5/dataset_generator/new_images_train/classes.txt'
filename = os.path.basename(source_file)
destination_file = os.path.join(output_dir, filename)
shutil.copy2(source_file, destination_file)

boxes = []

for i in range (1, 500):
    
    images = [filename for filename in os.listdir(directory) if filename.endswith('.png')]
    bgs = [filename for filename in os.listdir('bgs') if filename.endswith('.png')]
    
    selected_images = random.sample(images, 2)
    random_bg = Image.open(os.path.join('bgs', random.choice(bgs)))
    
    image_names = [os.path.splitext(image)[0] for image in selected_images]
    
    new_image = random_bg.resize((background_size, background_size))
    
    img1 = Image.open(os.path.join(directory, selected_images[0]))
    img1 = img1.convert("RGBA")
    img1 = transform(img1)
    
    scale_factor = random.uniform(0.9, 1)
    
    new_size = tuple(int(dim * scale_factor) for dim in img1.size)
    img1 = img1.resize(new_size)
    
    # Get the bounding box of the non-transparent part of the image
    bbox1 = img1.getbbox()

    # Calculate the width and height of the non-transparent part of the image at the given scale
    width = (bbox1[2] - bbox1[0])
    height = (bbox1[3] - bbox1[1])
    
    x1 = random.uniform(0, background_size - width)
    y1 = random.uniform(0, background_size - height)
    
    box1 = (x1, y1, x1 + width, y1 + height)
    boxes.append(box1)
    
    new_image.paste(img1, (int(x1-bbox1[0]), int(y1-bbox1[1])), img1)

    # Draw rectangles on new image based on box coordinates
    # draw = ImageDraw.Draw(new_image)
    # draw.rectangle(box1, outline="blue")
    
# draw the second box at a random position that does not overlap with the first box
    start_time = time.time()
    placed = False
    
    while not placed:
        
        img2 = Image.open(os.path.join(directory, selected_images[1]))
        img2 = img2.convert("RGBA")
        img2 = transform(img2)
        
        scale_factor = random.uniform(0.9, 1)
            
        new_size = tuple(int(dim * scale_factor) for dim in img2.size)
        img2 = img2.resize(new_size)
        
        # Get the bounding box of the non-transparent part of the image
        bbox2 = img2.getbbox()

        # Calculate the width and height of the non-transparent part of the image at the given scale
        width = (bbox2[2] - bbox2[0])
        height = (bbox2[3] - bbox2[1])
        
        x2 = random.uniform(0, background_size - width)
        y2 = random.uniform(0, background_size - height)
        
        box2 = (x2, y2, x2 + width, y2 + height)
        boxes.append(box2)

        if not any([intersect(box2, box1)]):
            
            new_image.paste(img2, (int(x2-bbox2[0]), int(y2-bbox2[1])), img2)

            # Draw rectangles on new image based on box coordinates
            # draw = ImageDraw.Draw(new_image)
            # draw.rectangle(box2, outline="red")
            
            break
        else:
            # print(f"{i} Retrying with another coordinate...2nd image")
            time.sleep(0.5)
            selected_images = random.sample(images, 2)
                
    new_image_name = f"{i}_{'_'.join(image_names)}_{folder_name}.jpg"
    new_image_path = os.path.join(output_dir, new_image_name)
    new_image.save(new_image_path)
    print(f"{new_image_name} saved")
    
    new_txt_name = f"{output_dir}/{i}_{'_'.join(image_names)}_{folder_name}.txt"
    
    labelimg_box = lblformat(box1, background_size)
    text1 = f"{detect_class(image_names[0])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"
    
    labelimg_box = lblformat(box2, background_size)
    text2 = f"{detect_class(image_names[1])} {labelimg_box[0]} {labelimg_box[1]} {labelimg_box[2]} {labelimg_box[3]}"

    with open(f"{new_txt_name}", "w") as file:
        file.write(text1 + '\n')
        file.write(text2 + '\n')